In [17]:
import csv
import os
import pandas as pd

compteur_sport_path = 'compteurs_sport.csv'
compteur_sup_path = 'compteur_enseignement_sup.csv'

data_es_full = 'data_es_full.csv'
enseignement_sup_full = 'enseignement_sup_full.csv'

rappro_operat = 'rappro_operat_1.csv'

operat_efa_path = 'operat_full_EFA.xlsx'

 # Récupérer les identifiants OPERAT sportif et enseignement sup ayant reçu un RNB ID lors du premier rapprochement

In [18]:
# Build the list of siret codes

target_siret = set()

with open(compteur_sport_path, "r") as f:
    for row in csv.DictReader(f, delimiter=";"):
        target_siret.add(row["CD_SIRT"])
        
with open(compteur_sup_path, "r") as f:
    for row in csv.DictReader(f, delimiter=";"):
        target_siret.add(row["CD_SIRT"])
        
print(f"Found {len(target_siret)} SIRET codes")

# Then use those siret codes to find "id_EFA"

target_efa = set() 

df = pd.read_excel(operat_efa_path)

reduced_operat = df[df["etablissement - etablissement_id → identifiant"].isin(target_siret)]

print(f"found {len(reduced_operat)}/{len(target_siret)} rows in the full Operat file")




Found 250 SIRET codes
found 387/250 rows in the full Operat file


In [21]:
# find some duplicates in the "etablissement - etablissement_id → identifiant" column

duplicates = reduced_operat[reduced_operat.duplicated("etablissement - etablissement_id → identifiant", keep=False)]

display(HTML(duplicates.to_html()))

,id_EFA,etablissement_occupant_type,etablissement - etablissement_id → identifiant,etablissement - etablissement_id → nom,occupant_siren_id,structure - occupant_siren_id → nom,structure - occupant_siren_id → identifiant,structure - occupant_siren_id → type,denomination EFA si SIREN occupant,complement_nom_EFA,adresse - adresse_id → commune_id,adresse - adresse_id → adresse,adresse - adresse_id → complement_adresse,adresse - adresse_id → latitude,adresse - adresse_id → longitude,adresse - adresse_id → numero_insee,commune - commune_id → code_postal,commune - commune_id → nom
77,207,SIRET,79823301100492,BASIC FIT II,NaN,NaN,NaN,NaN,NaN,NaN,71130.0,40 RUE DE LA VAGUE,4EME ETAGE HALL C,NaN,NaN,NaN,59650,VILLENEUVE D ASCQ
361,786,SIRET,79823301100492,BASIC FIT II,NaN,NaN,NaN,NaN,NaN,NaN,71130.0,40 RUE DE LA VAGUE,4EME ETAGE HALL C,NaN,NaN,NaN,59650,VILLENEUVE D ASCQ
725,1425,SIRET,77572257201091,C E SI,NaN,NaN,NaN,NaN,NaN,NaN,55093.0,AEROPORT MTPELLIER MEDITERRANEE,IMMEUBLE LE QUATRIEME,NaN,NaN,NaN,34130,MAUGUIO
932,1867,SIRET,19350097200016,INSTITUT NATIONAL DES SCIENCES APPLIQUEES DE RENNES,NaN,NaN,NaN,NaN,NaN,NaN,29668.0,20 AVENUE DES BUTTES DE COESMES,NaN,NaN,NaN,NaN,35700,RENNES
1008,1991,SIRET,77572257201174,CESI,NaN,NaN,NaN,NaN,NaN,NaN,72785.0,390 RUE CLAUDE NICOLAS LEDOUX,ZI AIX LES MILES IMMEUBLE LA CANOPEE,NaN,NaN,NaN,13290,AIX EN PROVENCE
1421,2876,SIRET,19660437500010,UNIVERSITE DE PERPIGNAN,NaN,NaN,NaN,NaN,NaN,UPVD PRESIDENCE RUE DU MUSEE,36307.0,52 AV PAUL ALDUY,NaN,NaN,NaN,NaN,66100,PERPIGNAN
1436,2891,SIRET,19660437500010,UNIVERSITE DE PERPIGNAN,NaN,NaN,NaN,NaN,NaN,UPVD CAMPUS MAILLY FONTAINE NEUVE SAINT SAUVEUR 1 ET 2,36307.0,52 AV PAUL ALDUY,NaN,NaN,NaN,NaN,66100,PERPIGNAN
1460,2916,SIRET,19660437500010,UNIVERSITE DE PERPIGNAN,NaN,NaN,NaN,NaN,NaN,IMMEUBLE DELACROIX ANCIENNE ECOLE MADAME ROLAND,36307.0,52 AV PAUL ALDUY,NaN,NaN,NaN,NaN,66100,PERPIGNAN
1607,3155,SIRET,31134913800017,GROUPE ESIEA,NaN,NaN,NaN,NaN,NaN,NaN,49011.0,38 RUE DOCTEURS CALMETTE ET GUERIN,NaN,48.085476,-0.756092,53130_1578_00038,53000,LAVAL
2847,5336,SIRET,79823301100492,BASIC FIT II,NaN,NaN,NaN,NaN,NaN,NaN,2988.0,29 RUE DU GENERAL LECLERC,NaN,49.895799,2.293578,80021_3760_00029,80000,AMIENS


In [9]:
# Get the first rappro and join the rnb ids using id_EFA
rappro_df = pd.read_csv(rappro_operat)


df = pd.merge(reduced_operat, rappro_df, on='id_EFA', how="inner")

print(f"We have {len(df)} rows in the merged DF")


We have 387 rows in the merged DF


In [15]:
# Count how many row have a rnb id
with_rnb_ids = df[(df['rnb_ids'].str.strip() != '[]')]
print(f"{len(with_rnb_ids)} have at least one RNB ID")

262 have at least one RNB ID


In [16]:
from IPython.display import display, HTML

# drop columns for display
with_rnb_ids_lighter = with_rnb_ids.drop(columns=[
    'occupant_siren_id', 
    'structure - occupant_siren_id → nom', 
    'structure - occupant_siren_id → identifiant',
    'structure - occupant_siren_id → type',
    'complement_nom_EFA',
    'adresse - adresse_id → commune_id',
    'adresse - adresse_id → latitude',
    'adresse - adresse_id → longitude',
    'adresse - adresse_id → numero_insee',
    'denomination EFA si SIREN occupant',
    'etablissement_occupant_type'
])


display(HTML(with_rnb_ids_lighter.to_html()))

,id_EFA,etablissement - etablissement_id → identifiant,etablissement - etablissement_id → nom,adresse - adresse_id → adresse,adresse - adresse_id → complement_adresse,commune - commune_id → code_postal,commune - commune_id → nom,rnb_ids,cas_assujetissement
1,372,39779265600027,YNCREA OUEST,20 RUE CUIRASSE BRETAGNE,NaN,29200,BREST,"['QAY4287VWAVH', 'QHNYYJP5TFC7']",1A
3,929,21500532300138,ACTIVITES DE LOISIRS,255 RUE DE LA MAIRIE,NaN,50380,SAINT PAIR SUR MER,['FWDBWVTE6BY4'],1A
4,1384,39779265600076,YNCREA OUEST,33 RUE DU CHAMP DE MANOEUVRE,NaN,44470,CARQUEFOU,['D5JCKV1CM2E3'],1A
6,1418,77572257200846,C E SI,93 BD DE LA SEINE,NaN,92000,NANTERRE,"['2TK86DN2993B', 'TW55ZYNWWA26', 'RV2DSD7HN7R7', '7CPQZ11RA8FA', '4NH9ZHEX7YFC']",1A
8,1420,77572257200978,C E SI,1 ALL DU TITANE,NaN,45100,ORLEANS,"['86CM4C9MMY8S', 'ATPBPBHPS9VG', 'KQACRE5RDS3F']",1A
10,1778,77572257200051,C E SI,7 RUE DIDEROT,NaN,62000,ARRAS,"['9FYEH3C48KYW', '4NDXZR3QV6AZ', 'JG23M31QV9X9', 'WTF67G6GR5QM', 'AA1CDGAFRMCR', '5TKG51Y3TR93', 'BYGZN7XEW15R', 'R91VSXJV7CQ8', 'JR9N91D8PWTB']",1A
12,1792,77572257200960,C E SI,1 AV AUGUSTIN LOUIS CAUCHY,NaN,44300,NANTES,['93RX7XRMB26F'],1A
13,1795,77572257201034,C E SI,264 BD GODARD,IMMEUBLE LE PHENIX,33300,BORDEAUX,['DVEWWJH6MJ67'],1B
14,1796,77572257200366,C E SI,16 RUE MAGELLAN,NaN,31670,LABEGE,['R8WQ4XWS3Z28'],1A
15,1867,19350097200016,INSTITUT NATIONAL DES SCIENCES APPLIQUEES DE RENNES,20 AVENUE DES BUTTES DE COESMES,NaN,35700,RENNES,"['J9TFDRKEGAF6', '99ZFP3VAMVKN', 'JD366HF61G1T', 'FTY8BTV15FKG', '5N929429KHQC', 'V9F7QV6T7KSC', '6CPWG7PJ4DXT', '8SKZFY9W7SGY', '16HVNJEC6ZR9', '6J4BCEC4XX4Q', 'N2K279E6C91N', '5DT61KZG9447', 'Y5TABX9JSWKH', '1P1W1D5TRS2H', 'ZNGXSV3FG78J', 'FYQ6BYGCW22Z', 'CQJQ7Q7ZNAHA', '2CYYWNXQZJ1X', 'X4XWARG7MDZZ', '8VJVPJHKJF6K', 'V8DBPGDA1VFM', '2V1F6PG3EDST', 'DVYFY77APD8R', '88Y38H877HG1', 'HR73DPY5FW31', 'J8766PC2KKEM', 'Z62SKPHSQSCQ', '52HWRBJ54GK4', '53BH5YWTAXH7', '65495PXBYHG8', 'MZCY2QR6HN6V', '1ZGTEQDJJNGV', '31G5W9G1NDCZ', 'YZCQR61TT36M', 'GJTWNSKJFVRX', 'NSXEKGNYQWNZ', '7SFBCBXHMTFS', '8C8JNSGRNNHX', 'M37YTFSJMGGE', '12KQ1ME5FHDN', 'PFV2AFWERFEB', '9HKNE4XQBR6M', '9DDNDPX4BJMH', 'AZSRQA8725TP', '4J2ZGNKFKPSR', 'C888ZYGZ6VN7', '18F2VNXAD5HW', 'NMWMK75EA7N4', '1NEGDGKEYJFB', 'HBBF2GF7B3A2', 'KTYFAXSVW95R', 'ZWJXN2FVSX3X']",3


On on a un problème, dans le cas de l'univesité de Perpignan (et surement dans d'autres cas), on a attaché des dizaines d'ID RNB (tous à la même adresse, celle de l'université) à une seule ligne EFA (et donc PDL).  Il est quasi certain qu'un seul PDL de fournit pas tous ces bâtiments.

On devrait quelle est la maille utilisée dans OPERAT pour cette université

In [24]:
df = pd.read_excel(operat_efa_path)

df = df[df["etablissement - etablissement_id → nom"] == "UNIVERSITE DE PERPIGNAN"]

display(HTML(df.to_html()))

,id_EFA,etablissement_occupant_type,etablissement - etablissement_id → identifiant,etablissement - etablissement_id → nom,occupant_siren_id,structure - occupant_siren_id → nom,structure - occupant_siren_id → identifiant,structure - occupant_siren_id → type,denomination EFA si SIREN occupant,complement_nom_EFA,adresse - adresse_id → commune_id,adresse - adresse_id → adresse,adresse - adresse_id → complement_adresse,adresse - adresse_id → latitude,adresse - adresse_id → longitude,adresse - adresse_id → numero_insee,commune - commune_id → code_postal,commune - commune_id → nom
1421,2876,SIRET,19660437500010,UNIVERSITE DE PERPIGNAN,NaN,NaN,NaN,NaN,NaN,UPVD PRESIDENCE RUE DU MUSEE,36307.0,52 AV PAUL ALDUY,NaN,NaN,NaN,NaN,66100,PERPIGNAN
1436,2891,SIRET,19660437500010,UNIVERSITE DE PERPIGNAN,NaN,NaN,NaN,NaN,NaN,UPVD CAMPUS MAILLY FONTAINE NEUVE SAINT SAUVEUR 1 ET 2,36307.0,52 AV PAUL ALDUY,NaN,NaN,NaN,NaN,66100,PERPIGNAN
1460,2916,SIRET,19660437500010,UNIVERSITE DE PERPIGNAN,NaN,NaN,NaN,NaN,NaN,IMMEUBLE DELACROIX ANCIENNE ECOLE MADAME ROLAND,36307.0,52 AV PAUL ALDUY,NaN,NaN,NaN,NaN,66100,PERPIGNAN
186470,277025,SIRET,19660437500010,UNIVERSITE DE PERPIGNAN,NaN,NaN,NaN,NaN,NaN,NaN,36307.0,52 AV PAUL ALDUY,NaN,NaN,NaN,NaN,66100,PERPIGNAN


On voit qu'il y a 4 EFA "UNIVERSITE DE PERPIGNAN". On voit aussi que la colonne "complement_nom_EFA" aurait pu nous aider à faire la distinction lors du rapprochement EnsSup.